# 反射

## 接口
任何类型只要实现 Read 或 Write 方法即实现 io.Reader 或 io.Writer 接口。意思就是：接口类型 io.Reader 可以被赋值任意实现 Read 方法的类型。
```
// Reader is the interface that wraps the basic Read method.
type Reader interface {
    Read(p []byte) (n int, err error)
}

// Writer is the interface that wraps the basic Write method.
type Writer interface {
    Write(p []byte) (n int, err error)
}
```

## The Laws of Reflection
- Reflection goes from interface value to reflection object.
- Reflection goes from reflection object to interface value.
- To modify a reflection object, the value must be settable.

In [ ]:
// 接口赋值，除了空接口还有这样的形式
func main() {
    var r io.Reader
    r = os.Stdin
    r = bufio.NewReader(r)
    r = new(bytes.Buffer)
    fmt.Printf("real obj: %#v", r)
}

In [ ]:
// 你也许会想接口在哪里，看起来只传递 float64 类型的 x 变量作为参数给 TypeOf 函数，而不是接口变量。
// 实际上 TypeOf 函数签名中的参数是空接口，x 会先赋值给空接口，然后作为函数参数传递，TypeOf 函数内部处理空接口恢复类型信息 Type。
func main() {
    var x float64 = 3.4
    fmt.Println("type:", reflect.TypeOf(x))
    fmt.Println("value:", reflect.ValueOf(x).String())
}

In [ ]:

type User struct {
    Id   int
    Name string
    Age  int
}

func (u User) ReflectCallFunc() {
    fmt.Println("Allen.Wu ReflectCallFunc")
}

func main() {

    user := User{1, "Allen.Wu", 25}
    DoFiledAndMethod(user)

}

// 通过接口来获取任意参数，然后一一揭晓
func DoFiledAndMethod(input interface{}) {
    getType := reflect.TypeOf(input)
    fmt.Println("get Type is :", getType.Name())
    getValue := reflect.ValueOf(input)
    fmt.Println("get all Fields is:", getValue)
    // 获取字段
    // 1. 先获取interface的reflect.Type，然后通过NumField进行遍历
    // 2. 再通过reflect.Type的Field获取其Field
    // 3. 最后通过Field的Interface()得到对应的value
    for i := 0; i < getType.NumField(); i++ {
        field := getType.Field(i)
        value := getValue.Field(i).Interface()
        fmt.Printf("%s: %v = %v\n", field.Name, field.Type, value)
    }
    // 获取方法
    // 1. 先获取interface的reflect.Type，然后通过.NumMethod进行遍历
    for i := 0; i < getType.NumMethod(); i++ {
        m := getType.Method(i)
        fmt.Printf("%s: %v\n", m.Name, m.Type)
    }
}